### Search-Askメソッド (あるいは Retrieval Augmented Generation) で GPT の回答精度を上げてみる

In [1]:
import os
import requests
import bs4
import chromadb
from chromadb.utils import embedding_functions
import openai
import pandas as pd

OCI資料活用集のページにある「Autonomous Database 技術FAQ」をスクレイピングして FAQ の質問と回答のセットを作成する (Pandas の Dataframe 形式)

In [2]:
faq_url = "https://oracle-japan.github.io/ocidocs/faq/services/autonomous/autonomous-database-faq/"
response = requests.get(faq_url)
soup = bs4.BeautifulSoup(response.text, 'html.parser')
df = pd.DataFrame(columns=['Q', 'A'])
for h2 in soup.findAll('h2'):
    question = h2.text.strip()
    l = []
    for n in h2.next_siblings:
        if type(n) is bs4.element.Tag:
            if n.name == 'h2':
                answer = (''.join(l)).replace('\n', '').strip()
                qa = pd.DataFrame({'Q': [question], 'A' : [answer]})
                df = pd.concat([df, qa], ignore_index=True)
                break
            if n.name == 'p':
                for d in n.contents:
                    if d.name == 'a':
                        l.append(f"{d.text}<{d.get('href')}>")
                    else:    
                        l.append(str(d.text))
df.to_csv('./adb-faq.csv', index=False)
df

,Q,A
0,Autonomous DatabaseはOracle Databaseとどう違うのでしょうか？,内部的にはOracle Database 19cを使用しています。(2023/8時点)それに...
1,Autonomous DatabaseではExadataのモデルを選択できますか？,Share型とDedicated型で異なります。Shared型ではモデルを選択することはでき...
2,Autonomous Database にはタイプがいくつかありますが、どれを選択すれば良い...,ワークロードの特性に応じてご選択いただければと思います。分析目的の利用であれば、Autono...
3,デプロイメント・タイプとしてSharedとDedicated、Cloud@Customerが...,Sharedは運用を極力シンプルにしたい場合や、コストを最優先に考えた場合に選択されることが...
4,SharedとDedicatedに機能差はありますか？,こちら<https://docs.oracle.com/en/cloud/paas/auto...
...,...,...
84,結果件数が多いクエリはどうすればよい？,「期待した性能が出ないんだけど、なぜ？」という問い合わせにおいて、比較的多いのがこのパターン...
85,インスタンスのキャッシュをクリアするにはどうしたらよいですか？,検証実施時にデータベースのキャッシュ（Buffer Cache）による性能向上効果を排除して...
86,パラレルクエリを利用して高速化したいのだが、どう設定すれば良い？？,接続サービスを選択いただきます。接続サービスに関してはこちら<https://oracle-...
87,INSERT処理を高速化するにはどうしたら良いですか？,接続サービスを選択いただきます。接続サービスの説明に関してはこちら<https://orac...


Embedding モデルを指定して ChromaDB (ベクトルデータベース) のコレクションを作成する

In [3]:
# OpenAI embedding
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
    api_key = os.getenv("OPENAI_API_KEY"),
    model_name="text-embedding-ada-002"
)

# Cohere https://docs.cohere.com/docs/supported-languages
cohere_english_ef = embedding_functions.CohereEmbeddingFunction(
    api_key = os.getenv("COHERE_API_KEY"),
    model_name="embed-english-v2.0"
)
cohere_multilingual_ef = embedding_functions.CohereEmbeddingFunction(
    api_key = os.getenv("COHERE_API_KEY"),
    model_name="embed-multilingual-v2.0"
)

# Chroma default embedding - Sentence Transformers all-MiniLM-L6-v2
default_ef = embedding_functions.DefaultEmbeddingFunction()


persist_directory="./chromadb-faq" # chromadb データベースファイルを保存するディレクトリ
embedding_function=openai_ef # ここでは OpenAI の embedding を使用

# faq コレクションの作成
client = chromadb.PersistentClient(path=persist_directory)
#client.delete_collection("faq")
collection = client.get_or_create_collection(name="faq", embedding_function=embedding_function)

In [4]:
# FAQのテキストは "Q:...\nA:..." のフォーマットにして embedding する
documents = ("Q: " + df['Q'] + "\nA: " + df['A']).to_list()

# FAQの各テキストに付与する id
ids = list(map(lambda x: f"adb-{x:03d}", df.index))

# FAQの各テキストに付与する メタデータ
metadatas = list(map(lambda x: {"category" : "adb"}, df['Q']))

# collection に FAQ を追加 embedding の生成は自動的に行われる
#collection.add(documents=documents, metadatas=metadatas, ids=ids)
collection.upsert(documents=documents, metadatas=metadatas, ids=ids)

In [5]:
# 検索のテスト
query = "バックアップの方式は？" 
results = collection.query(
        query_texts = query, 
        where = {"category" : "adb"}, 
        n_results = 5,
    )
results

{'ids': [['adb-028', 'adb-030', 'adb-027', 'adb-029', 'adb-035']],
 'distances': [[0.29316132787381893,
   0.33005908800307415,
   0.3307373417836906,
   0.3330326989339843,
   0.33692546481586033]],
 'metadatas': [[{'category': 'adb'},
   {'category': 'adb'},
   {'category': 'adb'},
   {'category': 'adb'},
   {'category': 'adb'}]],
 'embeddings': None,
 'documents': [['Q: 自動バックアップはどこに取得されるのでしょうか。任意のオブジェクト・ストレージに取得することはできますか？\nA: オラクル社が管理するオブジェクト・ストレージに取得されます。取得先に任意のオブジェクト・ストレージを指定することはできません。自動バックアップが取得されるオブジェクト・ストレージはユーザがアクセスすることはできず、課金対象外（ユーザーが指定するストレージ容量には含まれません）になります。',
   'Q: 自動バックアップからのリストア時間はどれくらいでしょうか？\nA: データ量、更新量に依存するため変動する可能性はありますが、サービスレベル目標についてはこちら<https://docs.oracle.com/en-us/iaas/autonomous-database-shared/doc/availability-slo.html>を参照ください。',
   'Q: バックアップは開始時点でスナップショットが取得されるのでしょうか？その場合、バックアップの完了を待たずともDDL/DML/起動停止等の操作は可能ですか。\nA: Autonomous Database は、ストレージ装置のスナップショット機能によるバックアップは採用しておらず、RMANを利用して60日ごとにフルバックアップ、週次で累積バックアップ、日次で増分バックアップを取得しております。増分バックアップの間の更新についてはアーカイブ・ログが取得

OpenAI を使って回答する関数を作成する (chat_completion(), chat_completion_with_query())

In [6]:
# 学習済の知識を使ってOpenAIが質問内容に回答する
def chat_completion(message):
    messages = [
        {"role": "system", "content": "Autonomous Database に関する質問に対する回答を行います。"},
        {"role": "user", "content": message},
    ]
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0
    )
    return response["choices"][0]["message"]["content"]

# 質問に関連する FAQ 項目を埋め込んだプロンプトを作る
def create_message(question, query_results):
    examples = []
    for n in range(0, len(query_results['documents'][0])):
        examples.append(f"\n{query_results['documents'][0][n]}\n")

    message = f"""
以下は Q&A です。
---
{''.join(examples)}
---
これまでの知識を使わず与えられた Q&A の情報だけを使って質問に回答して下さい。
回答できない場合は、"情報が不足しているためその質問に回答することができません。" と回答して下さい。
Q: {question}
A: 
"""
    return message

# 質問内容に関連するFAQを参照しながらOpenAIが回答する
def chat_completion_with_query(question, n_results=5, show_prompt=True):
    query_results = collection.query(
        query_texts = question, 
        where = {"category" : "adb"}, 
        n_results = n_results,
    )
    message = create_message(question, query_results)
    if show_prompt: print(message)
    return chat_completion(message)

二つの関数を実行して出力を比較する

In [7]:
print(chat_completion("接続数の制限はありますか？"))

はい、Autonomous Databaseには同時接続数の制限があります。制限は、データベースのエディションによって異なります。

Autonomous DatabaseのStandardエディションでは、最大100個の同時接続が許可されます。

Autonomous DatabaseのEnterpriseエディションでは、最大200個の同時接続が許可されます。

ただし、これらの制限はデータベースのパフォーマンスに影響を与える可能性があるため、適切な接続数を管理することが重要です。


In [8]:
print(chat_completion_with_query("接続数の制限はありますか？"))


以下は Q&A です。
---

Q: 一部の重たいクエリによるリソースの大量消費を防ぐ仕組みはありますか？
A: Runaway Queryの管理で、SQLの実行時間やIO量の上限を設定することができます。セッションは接続されたまま、上限を超えるクエリは強制キャンセルされます。参考：Runaway Query マニュアル<https://docs.oracle.com/en/cloud/paas/autonomous-database/adbsa/manage-sql-statements.html#GUID-4861BA7F-F9FA-4909-8DC0-4F46AFF80706>ページトップに戻る<#>

Q: Autonomous Databaseの同時接続セッション数はいくつでしょうか。また同時接続セッション数を超えて接続しようとした場合はエラーになりますか？
A: 同時接続セッション数は1OCPUあたり300セッションです。Autonomous Databaseの初期化パラメータSESSIONSとして設定されます。この値は接続サービスをまたいで有効であり、例えば、OCPU=1のAutonomous Transaction Processingにおいて、TPURGENTで100、TPで200セッションが接続している場合、新たにセッションを作成することはできません。セッション数を超えた場合は 「ORA-00018: 最大セッション数を超えました」というエラーとなり、セッションの作成に失敗します。なお、Autonomous DatabaseはOracle MultitenantのPluggable Databaseであるため、バックグランドプロセスや再帰セッション分のセッション数を考慮する必要はありません。

Q: パラレルクエリを利用して高速化したいのだが、どう設定すれば良い？？
A: 接続サービスを選択いただきます。接続サービスに関してはこちら<https://oracle-japan.github.io/ocitutorials/database/adb201-service-names/>を参照ください。

Q: INSERT処理を高速化するにはどうしたら良いですか？
A: 接続サービスを選択いただきます。接続サービスの説明に関してはこちら<http

FAQ にない質問をしてみる

In [9]:
print(chat_completion_with_query("地球と月の間の距離は？"))


以下は Q&A です。
---

Q: OCPU、ストレージの課金は時間単位でしょうか？
A: 価格表<https://www.oracle.com/jp/cloud/price-list.html#adw>にはストレージはTB/月、CPUはOCPU数/時で記載されていますが、 実際はどちらも秒単位の請求となります。OCPU、ストレージ共に最低1分から秒単位での利用が可能です。

Q: 自動バックアップからのリストア時間はどれくらいでしょうか？
A: データ量、更新量に依存するため変動する可能性はありますが、サービスレベル目標についてはこちら<https://docs.oracle.com/en-us/iaas/autonomous-database-shared/doc/availability-slo.html>を参照ください。

Q: 一時表領域のサイズは決められますか？
A: ストレージサイズの30%が最大サイズで自動拡張され、ユーザーは変更不可です。ページトップに戻る<#>

Q: ヒントは利用できますか？
A: おすすめの検証手順としては、まずはそのままの状態で計測してください。その後、予期した性能が出ない場合に限り、ヒントの効果を確認してください。

Q: SYSDATEのタイムゾーンがUTCですが、JSTに変更は可能でしょうか？
A: はい、SYSDATE_AT_DBTIMEZONEを使用することで可能です。以下の記事に手順が記載されていますのでご確認ください。[OCI]Autonomous Database：SYSDATE_AT_DBTIMEZONEを使用してsysdateが日本時間を返すようにしてみた<https://qiita.com/500InternalServerError/items/d741a7144de7b35e04ed>ページトップに戻る<#>

---
これまでの知識を使わず与えられた Q&A の情報だけを使って質問に回答して下さい。
回答できない場合は、"情報が不足しているためその質問に回答することができません。" と回答して下さい。
Q: 地球と月の間の距離は？
A: 



情報が不足しているためその質問に回答することができません。
